In [1]:
#https://www.incomebyzipcode.com/newyork/ scrape
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
zipcode_df = pd.read_csv('dataframe_with_zipcodes_only.csv', engine='python')
zipcode_df.head()

,neighborhood,zipcode
0,Alphabet City,10002.0
1,Arverne,11692.0
2,Astoria,11101.0
3,Astoria Heights,11370.0
4,Auburndale,11358.0


In [3]:
np.isnan(zipcode_df.zipcode[11])

True

In [4]:
#returns the zipcode URL
def get_url(zipcode):
    
    if np.isnan(zipcode) != True:
        return 'https://www.incomebyzipcode.com/newyork/'+str(zipcode)[0:5]
    else:
        return zipcode

In [22]:
get_url(zipcode_df.zipcode[1])

'https://www.incomebyzipcode.com/newyork/11692'

In [6]:
def get_page(zipcode):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    
    if np.isnan(zipcode) != True:
        url = get_url(zipcode)
        page = requests.get(url, headers=headers, timeout=50) #it will keep trying at this stage, the loop will not continue to iterate until a response is given
        c = page.content
        return BeautifulSoup(c, 'html.parser')
    
    else:
        return zipcode

In [26]:
get_page(zipcode_df.zipcode[1])

<!DOCTYPE html>

<html>
<head>
<title>11692 Income Statistics - Current Census Data for Zip Codes</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="Get the most current US Census income data for zip code 11692. Compare 11692 income statistics with data for neighboring New York zip codes, counties and cities." name="description"/>
<link href="https://www.incomebyzipcode.com/newyork/11692" rel="canonical"/>
<link data-turbolinks-track="false" href="https://d3u7142ylbwtzx.cloudfront.net/assets/application-b676cd5a8dc5e5047fddb62e61334ffac4a4f3d57480c8d2f3026e7887659191.css" media="all" rel="stylesheet">
<!-- Include Leaflet -->
<link href="https://unpkg.com/leaflet@1.2.0/dist/leaflet.css" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.6.3/css/all.css" integrity="sha384-LRlmVvLKVApDVGuspQFnRQJjkv0P7/YFrw84YYQtmYG4nK8c+M+NlmYDCv0rKWpG" rel="stylesheet"/>
<meta

In [25]:
#median income test
get_page(zipcode_df.zipcode[0]).find_all('div', class_ = 'container mt-5 px-3 px-md-0')[0].find('p').get_text().replace("\n          \nThe median household income ($", "").split(')')[0].replace(',', '')





'35449'

In [9]:
#High net income percentage
get_page(zipcode_df.zipcode[0]).find_all('div', class_ = 'container mt-5 px-3 px-md-0')[0].find_all('p')[3].get_text().split('%')[0]

'6.4'

In [10]:
def get_med_inc_percent_high(row_number):
    #sends the zipcode from the row
    page = get_page(zipcode_df.zipcode[row_number])
    
    if np.isnan(zipcode_df.zipcode[row_number]) != True:
        median_inc = page.find_all('div', class_ = 'container mt-5 px-3 px-md-0')[0].find('p').get_text().replace("\n          \nThe median household income ($", "").split(')')[0].replace(',', '')
    
        per_highinc = page.find_all('div', class_ = 'container mt-5 px-3 px-md-0')[0].find_all('p')[3].get_text().split('%')[0]
    
        return [zipcode_df.neighborhood[row_number],int(str(zipcode_df.zipcode[row_number])[0:5]),median_inc, per_highinc]
    else:
        return [zipcode_df.neighborhood[row_number],zipcode_df.zipcode[row_number],np.nan, np.nan]
    

In [11]:
get_med_inc_percent_high(116)

['SoHo', 10012, '104561', '22.7']

In [12]:
list(zipcode_df.neighborhood)

['Alphabet City',
 'Arverne',
 'Astoria',
 'Astoria Heights',
 'Auburndale',
 'Battery Park City',
 'Bay Ridge',
 'Bedford-Stuyvesant',
 'Bensonhurst',
 'Bergen Beach',
 'Beverley Square East',
 'Bithlo',
 'Boerum Hill',
 'Borough Park',
 'Bowery',
 'Briarwood',
 'Bronxwood',
 'Brooklyn Heights',
 'Bushwick',
 'Carnegie Hill',
 'Carroll Gardens',
 'Central Harlem',
 'Central Park',
 'Central Riverdale',
 'Central Slope',
 'Chelsea',
 'Chinatown',
 'Civic Center',
 'Clarion',
 'Clinton Hill',
 'Cobble Hill',
 'College Point',
 'Crown Heights',
 'DUMBO',
 'Ditmars',
 'Downtown Brooklyn',
 'Downtown Flushing',
 'East Flatbush',
 'East Harlem',
 'East New York',
 'East Village',
 'East Williamsburg',
 'Elmhurst',
 'Far Rockaway',
 'Financial District',
 'Flatbush',
 'Flatiron District',
 'Flushing',
 'Fordham Heights',
 'Fordham Manor',
 'Forest Hills',
 'Fort George',
 'Fort Greene',
 'Fresh Meadows',
 'Garment District',
 'Glendale',
 'Governors Island',
 'Gowanus',
 'Gramercy Park',
 'G

In [13]:
income_list = []

for row_number in range(0, len(list(zipcode_df.neighborhood))):
    income_dict={}
    income_dict['neighborhood'] = get_med_inc_percent_high(row_number)[0]
    income_dict['zipcode'] = get_med_inc_percent_high(row_number)[1]
    income_dict['median_income'] = get_med_inc_percent_high(row_number)[2]
    income_dict['percent_high_earner'] = get_med_inc_percent_high(row_number)[3]
    income_list.append(income_dict)


In [17]:
income_list[121]

{'neighborhood': 'Steinway',
 'zipcode': 11105,
 'median_income': '68050',
 'percent_high_earner': '6.8'}

In [18]:
zip_info_df = pd.DataFrame(income_list)

In [19]:
zip_info_df.head()

,median_income,neighborhood,percent_high_earner,zipcode
0,35449,Alphabet City,6.4,10002.0
1,45816,Arverne,3.0,11692.0
2,57866,Astoria,11.9,11101.0
3,59901,Astoria Heights,4.5,11370.0
4,65932,Auburndale,8.4,11358.0


In [21]:
zip_info_df.to_csv("zip_code_info.csv", index=False)